In [17]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_predict
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image

In [18]:
df = pd.read_csv('train_data_with_groups.csv')
df = df[df['Group'] == 2]
x_train, x_test, y_train, y_test = train_test_split(df.drop(['Bankrupt?','Group'],axis=1), df['Group'], train_size=0.8, random_state=42)

In [19]:
df = pd.read_csv('train_data_with_groups.csv')
# Split the data into features (X) and target (y)
X = df.drop('Bankrupt?', axis=1)
y = df['Bankrupt?']

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(random_state=42).fit(x_train, y_train)
model2 = RandomForestClassifier(random_state=42).fit(x_train, y_train)
model3 = LogisticRegression(random_state=42).fit(x_train, y_train)

model1_pred = model1.predict(x_test)
model2_pred = model2.predict(x_test)
model3_pred = model3.predict(x_test)

print('Model 1 accuracy:', accuracy_score(y_test, model1_pred))
print('Model 2 accuracy:', accuracy_score(y_test, model2_pred))
print('Model 3 accuracy:', accuracy_score(y_test, model3_pred))
print((accuracy_score(y_test, model1_pred)+accuracy_score(y_test, model2_pred)+accuracy_score(y_test, model3_pred))/3)
print(confusion_matrix(y_test, model1_pred))
print(confusion_matrix(y_test, model2_pred))
print(confusion_matrix(y_test, model3_pred))

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', LogisticRegression(random_state=42))
    ],
    final_estimator=RandomForestClassifier(random_state=43),
    cv = 10

)
print(df.shape[1])
stacking_clf.fit(x_train,y_train)

Model 1 accuracy: 0.9672977624784854
Model 2 accuracy: 0.9664371772805508
Model 3 accuracy: 0.9672977624784854
0.9670109007458406
[[1124    0]
 [  38    0]]
[[1117    7]
 [  32    6]]
[[1124    0]
 [  38    0]]
47


StackingClassifier(cv=10,
                   estimators=[('lr', LogisticRegression(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('svc', LogisticRegression(random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=43))

In [21]:
y_pred = stacking_clf.predict(x_test)

print('Model accuracy:',accuracy_score(y_test,y_pred))

confusion_matrix(y_test, y_pred)

Model accuracy: 0.9672977624784854


array([[1114,   10],
       [  28,   10]], dtype=int64)

Part 4

In [22]:
from sklearn.model_selection import cross_val_score


In [23]:
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline



# Create a pipeline that includes scaling and the classifier
clf = make_pipeline(StandardScaler(), SVC(kernel='poly', degree=3, coef0=1, gamma='scale'))

# Train the model
clf.fit(x_train, y_train)

# Predict using the trained model
y_pred = clf.predict(x_test)

# Calculate accuracy or other performance metrics
from sklearn.metrics import accuracy_score
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.9647160068846816


In [24]:
y_pred = cross_val_predict(stacking_clf, df.drop(['Bankrupt?'],axis=1), df['Bankrupt?'], cv=5)
print(accuracy_score(df['Bankrupt?'], y_pred))
confusion_matrix(df['Bankrupt?'], y_pred)

0.9621146891682453


array([[5551,   58],
       [ 162,   36]], dtype=int64)